### Adding The Comment Section and Read-Later Option:


Before we start, I want to convert my project view functions to class to practice what we have learned.

Your views.py file should look like the following block:

In [ ]:
from .models import Post
from django.views.generic import ListView,DetailView

class IndexView(ListView):
    model = Post
    template_name = "blog/index.html"
    context_object_name = "posts"
    queryset = Post.objects.all().order_by('-date')[:3]
    
class PostsView(ListView):
    model = Post
    template_name = "blog/all-posts.html"
    context_object_name = 'all_posts'
    queryset = Post.objects.all().order_by('-date')

class PostDetailView(DetailView):
    model = Post
    template_name = "blog/post-detail.html"
    context_object_name = "post"
    slug_field = "slug"
    slug_url_kwarg = "slug"

    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['post_tags'] = self.object.tags.all()
        return context

Your urls.py file should look like the following block:

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('', views.IndexView.as_view(), name='blog-starting-page'),
    path('posts/', views.PostsView.as_view(), name='blog-post-page'),
    path('posts/<slug:slug>', views.PostDetailView.as_view(), name='blog-post-detail')
]

Now, lets add the comments section:

1. Every post should have comments, and every user should be able to post a comment. This means that we need a new model called comment.
    - You can design the comment class the way you want
    - One thing to note is that this comment model should have a one to many relation with the post model! (a post can have many comments, but a comment belongs to one and only one post)

Your Comment class model should look like the following block:

In [ ]:
class Comment(models.Model):
    user_name = models.CharField(max_length=150)
    user_email = models.EmailField()
    text = models.TextField(max_length=500)
    post  = models.ForeignKey(Post, on_delete=models.CASCADE, related_name='comments')

Now, you need to makemigrations and then migrate<br>

2. Since I don't want to manually create the comment section input fields, lets define a form for it.

Your forms.py file should look like the following block:

In [ ]:
from django import forms
from .models import Comment

class CommentForm(forms.ModelForm):
    class Meta:
        model = Comment
        fields = ['user_name', 'user_email', 'text']
        labels = {
            "user_name":"Your Name",
            "user_email":"Your Email",
            "text":"Your Comment"
        }

Your post-detail.html template should look like the following block:

In [ ]:
{% extends "base.html" %}
{% load static %}

{% block title %}{{ post.title }}{% endblock %}

{% block css_files %}
    <link rel="stylesheet" href="{% static "blog/post-detail.css" %}">
{% endblock %}

{% block content %}
    <section id='summary'>
        <h2>{{ post.title }}</h2>
        <p>
            {% for tag in post_tags %}
                <span class='tag'>{{ tag.caption }} </span>
            {% endfor %}
        </p>
        <article>
            <img src="{{ post.image.url }}" alt="{{ post.title }}">
            <address>By <a href='mailto:{{ post.author.email }}'>{{ post.author }}</a></address>
            <div>
                Last Update on <time>{{ post.date|date:"d M Y" }}</time>
            </div>
        </article>
    </section>
    <main>
        <p>
            {{ post.content }}
        </p>
    </main>
    <div class='comments-section'>
        <form method="post">
            {% csrf_token %}
            <div class='top-div'>
                <div class='top-left'>
                    {{ form.user_name.label_tag }}
                    {% if form.user_name.errors %}
                        {{ form.user_name.errors}}
                    {% endif %}
                    {{ form.user_name }}
                    
                    {{ form.user_email.label_tag }}
                    {% if form.user_email.errors %}
                        {{ form.user_emaile.errors}}
                    {% endif %}
                    {{ form.user_email }}
                </div>
                <div class='top-right'>
                    <button type="submit">Submit Comment</button>
                </div>
            </div>
            <div class="bottom-div">
                {{ form.text.label_tag }}
                {% if form.text.errors %}
                        {{ form.text.errors}}
                    {% endif %}
                {{ form.text }}
            </div>
        </form>
    </div>
{% endblock %}

Your post-detail.css file should look like the following block:

In [ ]:
body{
    background-color:#e7e7e7;
}

#main-navigation{
    background: linear-gradient(to right top, #338dc4, #0bb5e9);
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.25);
}

#summary{
    margin: 8rem auto 3rem auto;
    padding:2rem;
    width:90%;
    max-width: 90rem;
    border-radius: 12px;
    background-color: #15753f;
    position: relative;
    box-shadow: 0 2px 8px rgba(0,0,0,0.45);
    height:22rem;
}

#summary h2{
    color:white;
    font-size:4.5rem;
    margin-bottom:-0.25rem;
}

#summary article{
    position:absolute;
    top:2rem;
    right: 3rem;
    padding: 1rem 2rem;
    border-radius: 12px;
    text-align: right;
    color: white;
}

#summary article img{
    width:12rem;
    height:12rem;
    border-radius: 12px;
    transform: rotate(5deg);
    border:5px solid white;
    margin-bottom: 1rem;
}

#summary:hover,
#summary:active{
    background-color:#338dc4;
}

#summary address{
    font-weight: bold;
}

#summary time{
    font-weight: bold;
}

#summary a{
    color:white;
}
#summary .tag{
    color:inherit;
    background-color: white;
    padding:0.3rem 1rem;
    border-radius: 8px;
    margin-left:5px;
}
main{
    margin:3rem auto 0 auto;
    background-color:white;
    width:90%;
    max-width: 90rem;
    padding:2rem;
    border-radius: 16px;
    box-shadow: 0 2px 12px rgba(0,0,0,0.45);
}
.comments-section{
    width:90%;
    max-width: 90rem;
    padding: 2rem;
    border-radius: 16px;
    background-color:rgba(209, 45, 20,0.8);
    margin:4rem auto 0 auto;
}
.comments-section form{
    width:50%;
    max-width: 50rem;
    margin:auto;
    padding:2rem;
    border:1px solid rgba(255,255,255,0.2);
    border-radius: 16px;
    box-shadow: 0px 0px 5px rgb(255,255,255);
    margin-top:2rem;
}
.top-div {
    display: flex;
    justify-content: space-between;
    align-items: center; /* Align items vertically centered */
}

.top-left {
    display: flex;
    flex-direction: column;
    margin-bottom: 0.5rem;
}
.top-left label{
    display:block;
    color:white;
}
.top-left input{
    width:30rem;
    margin-bottom:1rem;
    height:2rem;
    border-radius: 6px;
    border:2px solid gray;
}
.top-left input:hover,
.top-left input:focus{
    border-color:#338dc4;
}
.top-right button{
    width:8rem;
    height:8rem;
    border-radius: 8px;
    border:1px solid transparent;
    background-color:#338dc4;
    color:white;
}
.top-right button:hover,
.top-right button:focus{
    cursor: pointer;
    background-color:#0bb5e9;
}
.bottom-div{
    display: block;
    width:100%;
}
.bottom-div label{
    display:block;
    color:white;
}
.bottom-div textarea{
    width:100%;
    resize: none;
    height: 10rem;
    padding:1rem;
    font-size:1.2rem;
}

In the next notebook we will work on form submition.<br>